In [ ]:
import numpy as np
import pandas as pd
import os
from os import path
from tqdm import tqdm
from datetime import datetime

In [ ]:
# project_root
project_root = '/home/project/gas_anormaly_detection/restaurant/1training/'
data_root = path.join(project_root, 'Dataset/')
raw_restaurant_root = path.join(project_root, 'Dataset/restaurant/')
raw_restaurant_list = os.listdir(raw_restaurant_root)

## 1. segement data into weekly sequences

In [ ]:
week_cols = []
daysample_points =24
week_cols.extend(['pre'])
week_cols.extend(['Mon'] * daysample_points)
week_cols.extend(['Tues'] * daysample_points)
week_cols.extend(['Wed'] * daysample_points)
week_cols.extend(['Thur'] * daysample_points)
week_cols.extend(['Fri'] * daysample_points)
week_cols.extend(['Sat'] * daysample_points)
week_cols.extend(['Sun'] * daysample_points)


In [ ]:
# save weekly data and weekly date
meter_week_data_path = path.join(data_root, 'meter_week_data')
meter_week_date_path = path.join(data_root, 'meter_week_date')
if not os.path.exists(meter_week_data_path):
    os.mkdir(meter_week_data_path)
    print("creat new data dir.")

if not os.path.exists(meter_week_date_path):
    os.mkdir(meter_week_date_path)
    print("creat new date dir.")

In [ ]:
# find the first monday
def find_monday(date_data):
    i = 0
    while 1:
        date_string = date_data[i]
        date = datetime.strptime(date_string, '%Y-%m-%d %H:%M:%S')

        if date.weekday() == 0 and date_string.split(' ')[1]=='00:00:00':
            break
        else:
            i = i+1
    return date_string

In [ ]:
sample_num = 0

for f_name in tqdm(raw_restaurant_list):

    meter_week_data = pd.DataFrame(columns=week_cols)
    meter_week_date = pd.DataFrame(columns=week_cols)

    tmp_data = pd.read_csv(os.path.join(raw_restaurant_root, f_name))   

    id = os.path.splitext(f_name)[0]

    first_day = find_monday(tmp_data['Date'])

    first_index = tmp_data[tmp_data['Date'] == first_day].index.values[0]
    last_index = first_index + daysample_points * 7

    while (last_index < tmp_data.shape[0]):

        # save weekly data
        if first_index>0:
            append_data = tmp_data.iloc[first_index-1:last_index]['AccumFlow'].values
        else:
            append_data = np.append(np.array([np.nan]), tmp_data.iloc[first_index:last_index]['AccumFlow'].values)
        
        tmp_data_df = pd.DataFrame([append_data], columns=week_cols)
        meter_week_data = pd.concat([meter_week_data, tmp_data_df], ignore_index=True)

        # save weekly date
        if first_index>0:
            append_date = tmp_data.iloc[first_index-1:last_index]['Date'].values
        else:
            append_date = np.append(np.array([np.nan]), tmp_data.iloc[first_index:last_index]['Date'].values)
        
        tmp_date_df = pd.DataFrame([append_date], columns=week_cols)
        meter_week_date = pd.concat([meter_week_date, tmp_date_df], ignore_index=True)

        first_index = last_index
        last_index = first_index + daysample_points * 7
        sample_num+=1

    meter_week_data.to_csv(path.join(meter_week_data_path, f_name), index=False)
    meter_week_date.to_csv(path.join(meter_week_date_path, f_name), index=False)
    

## 2.  calculate gas consumption

In [ ]:
meter_week_data_list = os.listdir(meter_week_data_path)

In [ ]:
hourly_week_data_path = path.join(data_root, 'hourly_week_data')
hourly_week_date_path = path.join(data_root, 'hourly_week_date')
if not os.path.exists(hourly_week_data_path):
    os.mkdir(hourly_week_data_path)
    print("creat new data dir.")

if not os.path.exists(hourly_week_date_path):
    os.mkdir(hourly_week_date_path)
    print("creat new date dir.")


In [ ]:
seq_len=daysample_points*7

nan_delet_num =0
nan_delet_samples = []

sample_num_after_delet = 0 

for f_name in tqdm(meter_week_data_list):
    hourly_week_data = pd.DataFrame(columns=week_cols[1:])
    hourly_week_date = pd.DataFrame(columns=week_cols[1:])

    meter_week_data = pd.read_csv(os.path.join(meter_week_data_path, f_name))  
    meter_week_date = pd.read_csv(os.path.join(meter_week_date_path, f_name)) 
    for idx in range(meter_week_data.shape[0]):
        week_data= meter_week_data.iloc[idx,:].values
        week_date= meter_week_date.iloc[idx,:].values[1:]

        week_data[week_data<=0] = np.nan
        week_data = np.diff(week_data)
        
        if np.isnan(week_data).sum()>0:
            nan_delet_num+=1
            nan_delet_samples.append(week_data)
            continue

        sample_num_after_delet+=1
        week_data = pd.DataFrame([week_data], columns=week_cols[1:])
        week_date = pd.DataFrame([week_date], columns=week_cols[1:])

        hourly_week_data = pd.concat([hourly_week_data, week_data], ignore_index=True)
        hourly_week_date = pd.concat([hourly_week_date, week_date], ignore_index=True)
            
            

            
    hourly_week_data.to_csv(path.join(hourly_week_data_path, f_name), index=False)
    hourly_week_date.to_csv(path.join(hourly_week_date_path, f_name), index=False)
            
        

## 3. rule-based anomaly detection

In [ ]:
hourly_week_data_list = os.listdir(hourly_week_data_path)

In [ ]:
hourly_week_data_afterrules_path = path.join(data_root, 'hourly_week_data_afterrules')
hourly_week_date_afterrules_path = path.join(data_root, 'hourly_week_date_afterrules')
if not os.path.exists(hourly_week_data_afterrules_path):
    os.mkdir(hourly_week_data_afterrules_path)
    print("creat new data dir.")

if not os.path.exists(hourly_week_date_afterrules_path):
    os.mkdir(hourly_week_date_afterrules_path)
    print("creat new date dir.")


In [ ]:
zero_consumption_anomalies = []
negative_consumption_anomalies = []
burty_consumption_anomalies = []
sample_num_after_rules = 0

for f_name in tqdm(hourly_week_data_list):
    id_ = os.path.splitext(f_name)[0]
    hourly_week_data_preprocessed = pd.DataFrame(columns=week_cols)
    hourly_week_date_preprocessed = pd.DataFrame(columns=week_cols)

    hourly_week_data = pd.read_csv(os.path.join(hourly_week_data_path, f_name))  
    hourly_week_date = pd.read_csv(os.path.join(hourly_week_date_path, f_name)) 
    week_num = 0
    for idx in range(hourly_week_data.shape[0]):
        week_data= hourly_week_data.iloc[idx,:].values
        week_date= hourly_week_date.iloc[idx,:].values
            
        # rule1: zero consumption anomalies
        if (abs(week_data)<0.001).sum()==seq_len:
            zero_consumption_anomalies.append(id_)
            continue

        # rule2: negative consumption anomalies
        week_data_tep = week_data.copy()
        week_data_tep[week_data_tep>0.001]=0
        if np.nansum(week_data_tep)<0:
            negative_consumption_anomalies.append(id_)
            continue

        # rule3: burty consumption anomalies
        mean_usage = []
        for i in range(7):
            mean_usage.append(np.nanmax(week_data[i*24:i*24+24]))
        mean_usage = np.nanmean(mean_usage)

        if (week_data> max(5*mean_usage,10)).sum()>0:
            burty_consumption_anomalies.append(id_)
            continue
            
        sample_num_after_rules+=1
    
        
        week_data = pd.DataFrame([week_data], columns=week_cols)    
        week_date = pd.DataFrame([week_date], columns=week_cols)

        hourly_week_data_preprocessed = pd.concat([hourly_week_data_preprocessed, week_data], ignore_index=True)
        hourly_week_date_preprocessed = pd.concat([hourly_week_date_preprocessed, week_date], ignore_index=True)
        week_num+=1
            
    if week_num>0:
        hourly_week_data_preprocessed.to_csv(path.join(hourly_week_data_afterrules_path, f_name), index=False)
        hourly_week_date_preprocessed.to_csv(path.join(hourly_week_date_afterrules_path, f_name), index=False)
            
            